In [1]:
from pathlib import Path
import re

import srt

In [2]:
AutelSrtPath:Path = "./SampleData/MAX_0001.srt"
DjiSrtPath:Path = "./SampleData/DJI_0173.srt"

In [5]:
# Read all content and parse the srt content
with open(AutelSrtPath,"rt") as autelSrtFile:
    autelSrtString = autelSrtFile.read()

In [6]:
AutelSrtGenerator = srt.parse(autelSrtString)

In [7]:
Autel_content = AutelSrtGenerator.__next__()

In [8]:
Autel_content.content

'HOME(E: 121.498550, N: 24.601643) 2024-08-02 13:24:37\nGPS(E: 121.498672, N: 24.601662, 441.18m) \nISO:100 SHUTTER:30 EV:0.0 F-NUM:2.8 \nF.PRY (0.4°, -1.9°, 171.1°), G.PRY (0.0°, 0.0°, 171.2°)'

## DJI subtitle example
  
```
1
00:00:00,000 --> 00:00:01,000
F/2.8, SS 1667.11, ISO 100, EV 0, DZOOM 1.000, GPS (120.3534, 23.1201, 26), D 499.16m, H 98.00m, H.S 4.81m/s, V.S 0.10m/s 
```